In [53]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, dijkstra, format_path

from pprint import pprint
import numpy as np


In [54]:
def init_tamponi_per_reparto():
    tamponi_per_reparto = np.array([0, 0, 0, 3, 0, 0, 0, 6, 0, 0, 0, 0, 12, 0, 0, 0, 7, 0, 0, 0, 10, 0,
        0, 0, 3, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0]).astype(int)

    return tamponi_per_reparto

In [55]:
tamponi_per_reparto = init_tamponi_per_reparto()
print(tamponi_per_reparto)

[ 0  0  0  3  0  0  0  6  0  0  0  0 12  0  0  0  7  0  0  0 10  0  0  0
  3  0  1  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]


In [56]:
def estrai_reparti_con_tamponi(tamponi_per_reparto):
    return [i for i,t in enumerate(tamponi_per_reparto) if t > 0]

In [57]:
reparti_con_tamponi = estrai_reparti_con_tamponi(tamponi_per_reparto)

for index,node in enumerate(reparti_con_tamponi):
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha {tamponi_per_reparto[index]} tamponi")

0. Reparto con indice 3 e nome '4' ha 0 tamponi
1. Reparto con indice 7 e nome '7' ha 0 tamponi
2. Reparto con indice 12 e nome '11n' ha 0 tamponi
3. Reparto con indice 16 e nome '14' ha 3 tamponi
4. Reparto con indice 20 e nome '19' ha 0 tamponi
5. Reparto con indice 24 e nome '22' ha 0 tamponi
6. Reparto con indice 26 e nome '24' ha 0 tamponi
7. Reparto con indice 28 e nome '25/2' ha 6 tamponi


In [58]:
def compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto):
    reparti_con_tamponi = np.array(reparti_con_tamponi)
    score_matrix = np.zeros((len(reparti_con_tamponi), len(reparti_con_tamponi)))
    for i, indice_reparto in enumerate(reparti_con_tamponi):
        time = np.array(dijkstra(arc_matrix, indice_reparto))
        time = time[reparti_con_tamponi]
        score_matrix[i] = np.divide(tamponi_per_reparto[reparti_con_tamponi], time, out=-np.inf*np.ones_like(time), where=time!=0)
    return score_matrix

In [59]:
def retrieve_best_score(score_matrix, index):
    return np.argmax(score_matrix[index]), np.max(score_matrix[index])

In [60]:
score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)
for index,node in enumerate(reparti_con_tamponi):
    best_index, best_score = retrieve_best_score(score_matrix, index)
    print(f"{index}. Reparto con indice {node} e nome '{nodes[node]}' ha best score in reparto {best_index}. indice {reparti_con_tamponi[best_index]} con nome '{nodes[reparti_con_tamponi[best_index]]}' con score {best_score}")

0. Reparto con indice 3 e nome '4' ha best score in reparto 2. indice 12 con nome '11n' con score 0.6666666666666666
1. Reparto con indice 7 e nome '7' ha best score in reparto 2. indice 12 con nome '11n' con score 1.2
2. Reparto con indice 12 e nome '11n' ha best score in reparto 3. indice 16 con nome '14' con score 0.7
3. Reparto con indice 16 e nome '14' ha best score in reparto 2. indice 12 con nome '11n' con score 1.2
4. Reparto con indice 20 e nome '19' ha best score in reparto 2. indice 12 con nome '11n' con score 0.41379310344827586
5. Reparto con indice 24 e nome '22' ha best score in reparto 4. indice 20 con nome '19' con score 1.0
6. Reparto con indice 26 e nome '24' ha best score in reparto 1. indice 7 con nome '7' con score 1.2
7. Reparto con indice 28 e nome '25/2' ha best score in reparto 4. indice 20 con nome '19' con score 0.7142857142857143


In [61]:
def get_index_from_node(node, reparti_con_tamponi):
    try:
        index = reparti_con_tamponi.index(node)
    except:
        index = -1
        print("Nodo non trovato")
    return index

In [62]:
def get_next_node(start_node, reparti_con_tamponi, score_matrix):
    index = get_index_from_node(start_node, reparti_con_tamponi)
    next_index, _ = retrieve_best_score(score_matrix, index)
    score_matrix[:,index] = -np.inf
    return reparti_con_tamponi[next_index]

In [63]:
def stop_condition(residual_time):
    return residual_time <= 0 # aggiungere tempo al lab

In [69]:
def build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, current_node):
    path = []
    score_matrix = compute_score_matrix(arc_matrix, reparti_con_tamponi, tamponi_per_reparto)

    while not stop_condition(residual_time):
        next_node = get_next_node(current_node, reparti_con_tamponi, score_matrix)
        tempo_di_percorrenza = dijkstra(arc_matrix, current_node)[next_node]
        residual_time -= tempo_di_percorrenza
        print(f"Next node from {nodes[current_node]} is {nodes[next_node]} with time {tempo_di_percorrenza}. Residual time is {residual_time}")
        
        path.append(next_node)
        current_node = next_node
    
    return path

In [70]:
start_node = nodes.index('4')
residual_time = 120

path = build_tsp_path(arc_matrix, reparti_con_tamponi, tamponi_per_reparto, residual_time, start_node)
print(f"Path: {format_path(path)}")

Next node from 4 is 11n with time 18.0. Residual time is 102.0
Next node from 11n is 14 with time 10.0. Residual time is 92.0
Next node from 14 is 7 with time 12.0. Residual time is 80.0
Next node from 7 is 19 with time 19.0. Residual time is 61.0
Next node from 19 is 22 with time 10.0. Residual time is 51.0
Next node from 22 is 25/2 with time 18.0. Residual time is 33.0
Next node from 25/2 is 24 with time 24.0. Residual time is 9.0
Next node from 24 is 4 with time 17.0. Residual time is -8.0
Path: 11n -> 14 -> 7 -> 19 -> 22 -> 25/2 -> 24 -> 4


In [65]:
# TODO
# matrice di score tra start_node e i reparti con tamponi
# matrice di score tra i reparti con tamponi e laboratorio

start_node = nodes.index('X1')
index = get_index_from_node(start_node, reparti_con_tamponi)
index

Nodo non trovato


-1